In [1]:
import openmm.unit as u
import openmm.app as app
import openmm as mm
from sys import stdout

In [4]:
def run_simulation(prmtop_file, inpcrd_file, output_prefix):
    """
    Function to run a molecular dynamics simulation with OpenMM using AMBER input files.
    
    Parameters:
        prmtop_file (str): Path to the AMBER prmtop file.
        inpcrd_file (str): Path to the AMBER inpcrd file.
        output_prefix (str): Prefix for the output files (e.g., "simulation_output").
    """
    print("Loading AMBER input files...")
    # Load AMBER files
    prmtop = app.AmberPrmtopFile(prmtop_file)
    inpcrd = app.AmberInpcrdFile(inpcrd_file)
    
    print("Creating the system and integrator...")
    # Create a system and integrator
    system = prmtop.createSystem(nonbondedMethod=app.NoCutoff,
                                 constraints=app.HBonds,
                                 implicitSolvent=app.GBn2)
    integrator = mm.LangevinIntegrator(300 * u.kelvin, 1 / u.picosecond, 2 * u.femtoseconds)
    
    print("Initializing the simulation object...")
    # Create a simulation object
    simulation = app.Simulation(prmtop.topology, system, integrator)
    
    print("Setting initial positions...")
    # Set the initial positions
    simulation.context.setPositions(inpcrd.positions)
    
    print("Saving the topology to a PDB file...")
    # Save the topology to a PDB file
    topology_file = f"{output_prefix}_topology.pdb"
    with open(topology_file, 'w') as f:
        app.PDBFile.writeFile(simulation.topology, 
                              simulation.context.getState(getPositions=True).getPositions(), f)
    print(f"Topology saved to: {topology_file}")
    
    print("Minimizing the energy...")
    # Minimize the energy
    simulation.minimizeEnergy()
    print("Energy minimization complete.")
    
    print("Setting up reporters for the simulation...")
    # Set up reporters
    state_reporter = app.StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True)
    trajectory_file = f"{output_prefix}_trajectory.dcd"
    dcd_reporter = app.DCDReporter(trajectory_file, 100)
    
    simulation.reporters.append(state_reporter)
    simulation.reporters.append(dcd_reporter)
    
    print("Running the simulation for 100,000 steps...")
    # Run the simulation
    simulation.step(int(1E5))  # Run 100,000 steps
    print(f"Simulation complete. Trajectory saved to: {trajectory_file}")

    # Save minimized structure
    state = simulation.context.getState(getPositions=True)
    positions = state.getPositions()

    # Write minimized structure to the output PDB file
    output_pdb = f"{output_prefix}_minimized.pdb"
    
    with open(output_pdb, 'w') as f:
        app.PDBFile.writeFile(simulation.topology, positions, f)

    print(f"Energy minimization complete. Output saved to {output_pdb}")


# Example usage:
# run_simulation('/path/to/prmtop', '/path/to/inpcrd', 'output_prefix')


In [5]:
# Run simulation protein ligand system
# Takes about 50 minutes
run_simulation('/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/TrwD_atp.prmtop', '/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/TrwD_atp.inpcrd', 'TrwD_atp')

Loading AMBER input files...
Creating the system and integrator...
Initializing the simulation object...
Setting initial positions...
Saving the topology to a PDB file...
Topology saved to: TrwD_atp_topology.pdb
Minimizing the energy...
Energy minimization complete.
Setting up reporters for the simulation...
Running the simulation for 100,000 steps...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-42284.746337890625,258.56503770661124
2000,-40348.15234375,298.3293337855274
3000,-39952.57958984375,302.21012999915155
4000,-39732.93359375,294.27985492437716
5000,-39669.0498046875,308.1301599697121
6000,-40409.032958984375,297.60668616853155
7000,-39735.285400390625,301.87062791939803
8000,-39578.0966796875,298.52002570096874
9000,-39959.431640625,302.3368623840752
10000,-39667.838623046875,304.17646017932
11000,-40135.59619140625,300.6769077496016
12000,-40707.26123046875,307.7862837057738
13000,-40025.599365234375,304.30141317907174
14000,-39959.249755859375,299.16909811481